In [ ]:
!pip install tflearn

In [ ]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,GlobalMaxPooling1D,Flatten
from  tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [ ]:
%%writefile content.json
{"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Good day"],
     "responses": ["Hello, thanks for visiting KIIT university chatbot ", "Good to see you again friend", "Hi there, how can I help?"],
     "context_set": ""
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later, thanks for visiting KIIT university chatbot", "Have a nice day and keep hustling", "Bye! Come back again soon."]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful"],
     "responses": ["Happy to help students always!", "Any time!", "My pleasure"]
    },
    {"tag": "Fests / Cultural events",
     "patterns": ["What are the fests organized in the University?", "What are the societies?", "What are the Cultural fests here?" ],
     "responses": ["We have various clubs and societies like dance,singing,quiz,coding all registered under KSAC,KIIT fest is the largest organized fest of eastern india "]
    },
    {"tag": "location",
     "patterns": ["What is the univerity location?", "Where is KIIT located?", "What is your address?", "Where is the campus?" ],
     "responses": ["We have our one and only campus at Bhubaneswar,India with a massive area of 200 acres ", "We have our one and only campus at Bhubaneswar,India with a massive area of 25 sq KM"]
    },
    {"tag": "Computersciencedepartment",
     "patterns": ["How is the department of Computer Science?", "How is the computer science course in KIIT?", "Does KIIT provide the CS course in B.tech?" ],
     "responses": ["We have a reowned world class CS Faculty with some eminent teaching infrastructure."]
    },
    {"tag": "sports",
     "patterns": ["What is the sports infrastructure like in KIIT?", "Does KIIT provide sports facility to students?"],
     "responses": ["KIIT provide excellent sports facility , for a small example we have 8 gyms accross all the campuses and multiple swimming pools", "KIIT is a pioneer in the field of sports , for a small example we have 8 gyms accross all the campuses and multiple swimming pools"]
    },
    {"tag": "placements",
     "patterns": ["What the placement scenario at KIIT?", "What type of companies visit KIIT during placements?"],
     "responses": ["Our students crack top-knotch companies in placements, companies come in 3 rounds Day-0,Day-1 and Day-2.The highest offer was made by Yugabite for 53 LPA"],
     "context_set": "jobs"
    },
    {"tag": "Exams",
     "patterns": ["What is the exam structure like?", "What is the grading ctriteria?"],
     "responses": ["We have a efficient grading system which helps in proper academic evaluation of all students."],
     "context_filter": "exam"
    },
    {"tag": "hostel",
     "patterns": ["Is hostel accomodation available for students?", "What type of hostels are available?"],
     "responses": ["All type of hostels including AC,Non-AC .We have all type of rooms including 3-bed,2-bed with attached bathrooms"],
     "context_filter": "hostel"
    },
    {
      "tag":"number1",
      "patterns":["61279 42001"],
      "responses": ["200011"]
    },
    {
      "tag":"number2",
      "patterns":["61279 42002"],
      "responses": ["200012"]
    },
    {
      "tag":"number3",
      "patterns":["73211 67811"],
      "responses": ["200015"]

    },
    {
      "tag":"number4",
      "patterns":["73211 67812"],
      "responses": ["200016"]

    }

]
}

In [ ]:
#importing the dataset
with open('content.json') as content:
  data1=json.load(content)

In [ ]:
#getting all data to lists
tags=[]
inputs=[]
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [ ]:
#converting to dataframe
data=pd.DataFrame({"inputs":inputs,"tags":tags})

In [ ]:
data=data.sample(frac=1)

In [ ]:
#removing punctuations
import string
data['inputs']=data['inputs'].apply(lambda wrd:[lrts.lower() for lrts in wrd if lrts not in string.punctuation])
data['inputs']=data['inputs'].apply(lambda wrd: ''.join(wrd))
data

In [ ]:
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train=tokenizer.texts_to_sequences(data['inputs'])
#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train=pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(data['tags'])

In [ ]:
input_shape=x_train.shape[1]
print(input_shape)

In [ ]:
#define vocabulary
vocabulary=len(tokenizer.word_index)
print("no of unique words :",vocabulary)
output_length=le.classes_.shape[0]
print("output length")

In [ ]:
#creating the model

i=Input(shape=(input_shape))
x=Embedding(vocabulary+1,10)(i)
x=LSTM(10,return_sequences=True)(x)
x=Flatten()(x)
x=Dense(output_length,activation="softmax")(x)
model=Model(i,x)

In [ ]:
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
#training the model
train=model.fit(x_train,y_train,epochs=200)

In [ ]:
#plotting model accuracy
plt.plot(train.history['acc'],label='training set accuracy')
plt.plot(train.history['loss'],label='training set loss')
plt.legend()

In [ ]:
#chatting
import random

while True:
  texts_p=[]
  prediction_input=input('You : ')
  
  #removing punctuation and converting to upper case
  prediction_input=[letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input=''.join(prediction_input)
  texts_p.append(prediction_input)

  #tokenizing and padding
  prediction_input=tokenizer.texts_to_sequences(texts_p)
  prediction_input=np.array(prediction_input).reshape(-1)
  prediction_input=pad_sequences([prediction_input],input_shape)

  #getting output from model
  output=model.predict( prediction_input)
  output=output.argmax()

  #finding the right tag and predicting
  response_tag=le.inverse_transform([output])[0]
  print("KIIT Help Bot : ",random.choice(responses[response_tag]))
  if (response_tag == "goodbye") :
    break

In [ ]:
import pickle

In [ ]:
filename='chatbot_trained.sav'
pickle.dump(model,open(filename,'wb'))